#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [2]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [3]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [4]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 2*365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [7]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["OPTIEMUS.NS","TBZ.NS","DCBBANK.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-10-26 20:39:13,051 - INFO - 📈 Fetching data for 2178 tickers...
2025-10-26 20:39:13,055 - INFO - Fetching data from 2023-09-27 to 2025-10-26


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-10-26 20:40:42,706 - ERROR - 
2 Failed downloads:
2025-10-26 20:40:42,708 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-10-26 20:40:42,709 - ERROR - ['AHLEAST.NS']: Timeout('Failed to perform, curl: (28) Operation timed out after 10008 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
2025-10-26 20:40:47,357 - INFO - ✅ Download completed in 94.30 seconds
2025-10-26 20:40:52,111 - INFO - 📊 Final dataset: 1,027,367 rows from 2023-09-27 to 2025-10-24


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-09-27,20MICRONS.NS,NaN,128.239243,130.664451,126.704922,128.684687,17643.0,128.239243,128.239243,128.239243,128.239243,NaN,NaN,20MICRONS
1,2023-09-28,20MICRONS.NS,NaN,126.111008,129.674582,124.774668,124.774668,17952.0,128.036554,128.155782,128.197099,128.218066,NaN,NaN,20MICRONS
2,2023-09-29,20MICRONS.NS,NaN,132.099792,132.396760,124.725172,124.725172,61110.0,128.423529,128.310449,128.274380,128.256690,NaN,NaN,20MICRONS
3,2023-10-03,20MICRONS.NS,NaN,138.682510,138.682510,126.259485,131.703837,167442.0,129.400575,128.717197,128.480482,128.360430,NaN,NaN,20MICRONS
4,2023-10-04,20MICRONS.NS,NaN,145.611694,145.611694,135.118936,141.404695,266672.0,130.944491,129.379726,128.819714,128.532084,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [8]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,CENTRALBK,2025-10-24,140672.7339,80.0,1.22,58.09,True,37.930796,37.399382,40.921827,39.07,1.296222e+07,7.975041e+06
1,COASTCORP,2025-10-24,53964.9821,80.0,1.59,75.57,True,37.591339,36.280451,39.155799,42.84,9.384134e+05,3.433010e+05
2,DIAMONDYD,2025-10-24,25613.4288,80.0,0.81,58.28,True,1031.158168,1017.885954,1033.419580,1037.50,1.380521e+05,9.066520e+04


#### EMA Signal indicator - TBC

##### Update the Stock returns table

In [9]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 1887
✅ Total signals after merging: 314778
📦 Existing SIGNAL_RETURNS records: 0
✅ New unique records found: 1865
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
0,20MICRONS,Golden cross,2025-07-17,254.0,6.43,270.329987,2025-07-24,-7.69,234.470001,2025-07-31,-10.47,227.399994,2025-08-18,-14.98,215.960007,2025-10-15,NaN,NaN,NaT,NaN,NaN,NaT,-17.12,210.509995,2025-10-24,6.43,-7.69,-10.47,-14.98,<NA>,<NA>,-17.12
1,20MICRONS,Golden cross,2025-07-16,259.0,6.86,276.779999,2025-07-23,-5.46,244.850006,2025-07-30,-12.20,227.399994,2025-08-18,-17.18,214.500000,2025-10-14,NaN,NaN,NaT,NaN,NaN,NaT,-18.72,210.509995,2025-10-24,6.86,-5.46,-12.2,-17.18,<NA>,<NA>,-18.72
2,20MICRONS,Golden cross,2025-06-17,237.0,-4.12,227.243713,2025-06-24,-2.58,230.897217,2025-07-01,7.12,253.863495,2025-07-17,-4.95,225.270004,2025-09-15,NaN,NaN,NaT,NaN,NaN,NaT,-11.18,210.509995,2025-10-24,-4.12,-2.58,7.12,-4.95,<NA>,<NA>,-11.18
3,360ONE,Golden cross,2025-06-12,1081.0,3.09,1114.400024,2025-06-19,7.76,1164.849976,2025-06-26,9.92,1188.199951,2025-07-14,-0.28,1078.000000,2025-09-10,NaN,NaN,NaT,NaN,NaN,NaT,9.69,1185.699951,2025-10-24,3.09,7.76,9.92,-0.28,<NA>,<NA>,9.69
4,3MINDIA,Accumulation Signal,2025-08-07,32000.0,-5.16,30350.000000,2025-08-14,-3.52,30875.000000,2025-08-21,-3.64,30835.000000,2025-09-08,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-7.08,29735.000000,2025-10-24,-5.16,-3.52,-3.64,<NA>,<NA>,<NA>,-7.08
